In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile2 = pd.read_pickle("BBB_Train_Decile2.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile2.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile2, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 6s 2ms/step - loss: 0.5010 - acc: 0.7781 - val_loss: 0.4769 - val_acc: 0.7772
Epoch 36/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5004 - acc: 0.7765 - val_loss: 0.4765 - val_acc: 0.7759
Epoch 37/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5031 - acc: 0.7813 - val_loss: 0.4765 - val_acc: 0.7785
Epoch 38/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4976 - acc: 0.7771 - val_loss: 0.4755 - val_acc: 0.7759
Epoch 39/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4973 - acc: 0.7787 - val_loss: 0.4755 - val_acc: 0.7785
Epoch 40/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5043 - acc: 0.7749 - val_loss: 0.4752 - val_acc: 0.7759
Epoch 41/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4995 - acc: 0.7749 - val_loss: 0.4747 - val_acc: 0.7811
Epoch 42/75
3087/3087 [==============================] - 5s 2ms/ste

3087/3087 [==============================] - 5s 2ms/step - loss: 0.4669 - acc: 0.7924 - val_loss: 0.4657 - val_acc: 0.7889
Epoch 20/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4782 - acc: 0.7868 - val_loss: 0.4652 - val_acc: 0.7902

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 21/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4718 - acc: 0.7891 - val_loss: 0.4652 - val_acc: 0.7902
Epoch 22/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4693 - acc: 0.7907 - val_loss: 0.4651 - val_acc: 0.7902
Epoch 23/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4753 - acc: 0.7898 - val_loss: 0.4651 - val_acc: 0.7902
Epoch 24/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4766 - acc: 0.7891 - val_loss: 0.4650 - val_acc: 0.7902
Epoch 25/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4744 - acc: 0.7894 - val_loss: 0.4650 - v

3087/3087 [==============================] - 6s 2ms/step - loss: 0.5603 - acc: 0.7373 - val_loss: 0.5103 - val_acc: 0.7591
Epoch 4/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5386 - acc: 0.7603 - val_loss: 0.4953 - val_acc: 0.7694
Epoch 5/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5229 - acc: 0.7710 - val_loss: 0.4908 - val_acc: 0.7811
Epoch 6/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5144 - acc: 0.7668 - val_loss: 0.4905 - val_acc: 0.7772
Epoch 7/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5047 - acc: 0.7768 - val_loss: 0.4805 - val_acc: 0.7811
Epoch 8/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5044 - acc: 0.7710 - val_loss: 0.4831 - val_acc: 0.7759
Epoch 9/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4960 - acc: 0.7797 - val_loss: 0.4830 - val_acc: 0.7824
Epoch 10/100
3087/3087 [==============================] - 5s 2ms/st

3087/3087 [==============================] - 5s 2ms/step - loss: 0.4871 - acc: 0.7817 - val_loss: 0.4731 - val_acc: 0.7863
Epoch 63/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4830 - acc: 0.7859 - val_loss: 0.4732 - val_acc: 0.7850
Epoch 64/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4811 - acc: 0.7856 - val_loss: 0.4731 - val_acc: 0.7850
Epoch 65/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4807 - acc: 0.7826 - val_loss: 0.4731 - val_acc: 0.7850
Epoch 66/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4763 - acc: 0.7852 - val_loss: 0.4731 - val_acc: 0.7850
Epoch 67/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4875 - acc: 0.7843 - val_loss: 0.4732 - val_acc: 0.7850
Epoch 68/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4842 - acc: 0.7862 - val_loss: 0.4731 - val_acc: 0.7863
Epoch 69/100
3087/3087 [==============================] - 5s 

Epoch 22/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4840 - acc: 0.7830 - val_loss: 0.4677 - val_acc: 0.7863
Epoch 23/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4886 - acc: 0.7762 - val_loss: 0.4695 - val_acc: 0.7798
Epoch 24/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4886 - acc: 0.7794 - val_loss: 0.4665 - val_acc: 0.7850
Epoch 25/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4847 - acc: 0.7888 - val_loss: 0.4653 - val_acc: 0.7863
Epoch 26/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4787 - acc: 0.7927 - val_loss: 0.4645 - val_acc: 0.7927
Epoch 27/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4764 - acc: 0.7826 - val_loss: 0.4694 - val_acc: 0.7798
Epoch 28/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4799 - acc: 0.7846 - val_loss: 0.4660 - val_acc: 0.7850
Epoch 29/75
3087/3087 [==============================] 

3087/3087 [==============================] - 5s 2ms/step - loss: 0.5382 - acc: 0.7554 - val_loss: 0.5074 - val_acc: 0.7642
Epoch 12/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5328 - acc: 0.7613 - val_loss: 0.5023 - val_acc: 0.7668
Epoch 13/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5352 - acc: 0.7632 - val_loss: 0.4994 - val_acc: 0.7694
Epoch 14/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5339 - acc: 0.7603 - val_loss: 0.4955 - val_acc: 0.7785
Epoch 15/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5243 - acc: 0.7619 - val_loss: 0.4923 - val_acc: 0.7772
Epoch 16/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5280 - acc: 0.7684 - val_loss: 0.4908 - val_acc: 0.7772
Epoch 17/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5232 - acc: 0.7677 - val_loss: 0.4892 - val_acc: 0.7759
Epoch 18/100
3087/3087 [==============================] - 6s 

Epoch 23/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5329 - acc: 0.7690 - val_loss: 0.4923 - val_acc: 0.7694
Epoch 24/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5286 - acc: 0.7700 - val_loss: 0.4918 - val_acc: 0.7668
Epoch 25/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5194 - acc: 0.7668 - val_loss: 0.4899 - val_acc: 0.7694
Epoch 26/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5189 - acc: 0.7694 - val_loss: 0.4873 - val_acc: 0.7733
Epoch 27/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5178 - acc: 0.7784 - val_loss: 0.4867 - val_acc: 0.7694
Epoch 28/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5161 - acc: 0.7729 - val_loss: 0.4857 - val_acc: 0.7681
Epoch 29/125
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5192 - acc: 0.7749 - val_loss: 0.4847 - val_acc: 0.7733
Epoch 30/125
3087/3087 [========================

3087/3087 [==============================] - 3s 1ms/step - loss: 0.4854 - acc: 0.7872 - val_loss: 0.4671 - val_acc: 0.7746

Epoch 00028: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 29/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4823 - acc: 0.7862 - val_loss: 0.4665 - val_acc: 0.7772
Epoch 30/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4720 - acc: 0.7894 - val_loss: 0.4658 - val_acc: 0.7759
Epoch 31/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4750 - acc: 0.7894 - val_loss: 0.4653 - val_acc: 0.7772
Epoch 32/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4776 - acc: 0.7956 - val_loss: 0.4647 - val_acc: 0.7759
Epoch 33/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4815 - acc: 0.7856 - val_loss: 0.4645 - val_acc: 0.7772

Epoch 00033: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 34/50
3087/3087 [==============================] 

In [ ]:
0.8297